# Core Objects in VMC

This notebook demonstrates VMC core objects using ApoE alleles, haplotypes, and genotypes.

The ApoE gene is associated with risks of Alzheimer's disease and hypercholesterolemia. Risk of AD is attributed to haplotypes comprised of two locations, [rs429358](https://www.ncbi.nlm.nih.gov/snp/rs429358) and [rs7412](https://www.ncbi.nlm.nih.gov/snp/rs7412), both of which are C/T transitions.  The four ApoE haplotypes are defined by the two states (C and T) at the two locations shown below.

```
                             rs7412 
                             NC_000019.10:g.44908822
                             NM_000041.3:c.526
                             C          T
rs429358                 C   APOE-ε4    APOE-ε1
NC_000019.10:g.44908684  T   APOE-ε3    APOE-ε2
NM_000041.3:c.388
```

Links:

https://ghr.nlm.nih.gov/gene/APOE

[APO E4](https://www.ncbi.nlm.nih.gov/clinvar/variation/441269/)

http://snpedia.com/index.php/APOE

## Setup

In [1]:
# Use these strings as the namespaces for ApoE haplotypes and dbSNP locations
APOE_NAMESPACE = "ApoE"
DBSNP_NAMESPACE = "dbSNP"


import collections
import datetime
import json
import os

import jsonschema

from vmc import models, computed_id, serialize, schema_path
from vmc.utils import id_to_ir, ir_to_id, get_vmc_sequence_identifier

# pretty print json
def opp(o): print(json.dumps(json.loads(o.serialize()), indent=4, sort_keys=True))

## Identifiers
The `identifers` map is a map of internal ids to a list of external identifiers.
Object relationships will be defined in terms of internal ids.
The external identifiers enable others to interpret 

In [2]:
identifiers = collections.defaultdict(set)

## Sequences
A description of sequence variation, with VMC or otherwise, requires the availability of sequences in order to define coordinate systems.  Typically sequences are referred to with an accession like NC_000019.10.  There are two issues with using sequence accessions:

* Identical sequences have different names (e.g., "NC_000019.10" == "CM000681.2" == (GRCh38) "19" == (GRCh38 UCSC) "chr19").  Naive comparison of the same allele defined using different sequence name will fail.
* With graph genomes, it will become infeasible to assign sequence identifiers.

For these reasons, VMC encourages (but doesn't require) the use of computed identifiers based on a SHA512 digest, truncated to 24 bytes, and URL-safe base64 encoded.

get_vmc_sequence_id returns the computed sequence identifier for a given accession.

In [3]:
ir = "RefSeq:NC_000019.10"
sequence_ir = get_vmc_sequence_identifier(ir)
sequence_ir

'VMC:GS_IIB53T8CNeJJdUqzn9V_JnRtQadwWCbl'

In [4]:
identifiers[sequence_ir].add(ir)

## Intervals and Locations
An Interval consists of start and end positions in interbase coordinates.
A Location refers to a continuous span within a sequence, where the sequence is identified by Id and the span is defined by an Interval.

In [5]:
locations_by_name = {
    "rs429358": models.Location(
        sequence_id = sequence_ir,
        interval = models.Interval(start=44908683, end=44908684),
    ),
    "rs7412": models.Location(
        sequence_id = sequence_ir,
        interval=models.Interval(start=44908821, end=44908822),
    )
}
for n, l in locations_by_name.items():
    l.id = computed_id(l)
    identifiers[l.id].add(ir_to_id(models.Identifier(namespace=DBSNP_NAMESPACE, accession=n)))

In [6]:
# This is the string that is hashed to generate a computed identifier
serialize(locations_by_name["rs429358"])

'<Location|VMC:GS_IIB53T8CNeJJdUqzn9V_JnRtQadwWCbl|<Interval|44908683|44908684>>'

In [7]:
opp(locations_by_name["rs429358"])

{
    "id": "VMC:GL_L1IS6jOwSUsOpKihGRcqxHul1IwbV-1s",
    "interval": {
        "end": 44908684,
        "start": 44908683
    },
    "sequence_id": "VMC:GS_IIB53T8CNeJJdUqzn9V_JnRtQadwWCbl"
}


## Alleles

In [8]:
alleles_by_name = {
    "rs429358T": models.Allele(location_id=locations_by_name["rs429358"].id, state="T"),
    "rs429358C": models.Allele(location_id=locations_by_name["rs429358"].id, state="C"),
    "rs7412T":   models.Allele(location_id=locations_by_name["rs7412"].id,   state="T"),
    "rs7412C":   models.Allele(location_id=locations_by_name["rs7412"].id,   state="C"),
}
for n, a in alleles_by_name.items():
    a.id = computed_id(a)
    identifiers[a.id].add(ir_to_id(models.Identifier(namespace=DBSNP_NAMESPACE, accession=n)))

In [9]:
serialize(alleles_by_name["rs429358C"])

'<Allele|VMC:GL_L1IS6jOwSUsOpKihGRcqxHul1IwbV-1s|C>'

In [10]:
opp(alleles_by_name["rs429358C"])

{
    "id": "VMC:GA_zsJuMckKGajqHCl16sxKQJtBMjGrFHHZ",
    "location_id": "VMC:GL_L1IS6jOwSUsOpKihGRcqxHul1IwbV-1s",
    "state": "C"
}


## Haplotypes

In [11]:
haplotypes_by_name = {
    "ε1": models.Haplotype(
        allele_ids = [alleles_by_name["rs429358C"].id, alleles_by_name["rs7412T"].id],
        completeness = "COMPLETE"
    ),
    "ε2": models.Haplotype(
        allele_ids = [alleles_by_name["rs429358T"].id, alleles_by_name["rs7412T"].id],
        completeness = "COMPLETE"
    ),
    "ε3": models.Haplotype(
        allele_ids = [alleles_by_name["rs429358T"].id, alleles_by_name["rs7412C"].id],
        completeness = "COMPLETE"
    ),
    "ε4": models.Haplotype(
        allele_ids = [alleles_by_name["rs429358C"].id, alleles_by_name["rs7412C"].id],
        completeness = "COMPLETE"
    ),
}

for n, h in haplotypes_by_name.items():
    h.id = computed_id(h)
    identifiers[h.id].add(ir_to_id(models.Identifier(namespace=APOE_NAMESPACE, accession=n)))

In [12]:
serialize(haplotypes_by_name["ε1"])

'<Haplotype||COMPLETE|[VMC:GA_BqFKgQ350SUi4-uaGCvMM1ag7z0dNmvC;VMC:GA_zsJuMckKGajqHCl16sxKQJtBMjGrFHHZ]>'

In [13]:
opp(haplotypes_by_name["ε4"])

{
    "allele_ids": [
        "VMC:GA_zsJuMckKGajqHCl16sxKQJtBMjGrFHHZ",
        "VMC:GA__8rLiy7YkQDNy-t536RpVFGxIDiWLr6J"
    ],
    "completeness": "COMPLETE",
    "id": "VMC:GH_xk_4sKZKfwD7ol3H89mDShrBT3dfu5Aq"
}


In [14]:
# Reversing allele ids results in the same digest (that's good!)
h_ε4r = models.Haplotype(
        allele_ids = [alleles_by_name["rs7412C"].id, alleles_by_name["rs429358C"].id],
        completeness = "COMPLETE"
)
h_ε4r.id = computed_id(h_ε4r)
opp(h_ε4r)

{
    "allele_ids": [
        "VMC:GA__8rLiy7YkQDNy-t536RpVFGxIDiWLr6J",
        "VMC:GA_zsJuMckKGajqHCl16sxKQJtBMjGrFHHZ"
    ],
    "completeness": "COMPLETE",
    "id": "VMC:GH_xk_4sKZKfwD7ol3H89mDShrBT3dfu5Aq"
}


## Genotypes

In [15]:
genotypes_by_name = {
    "ε2/ε3": models.Genotype(
        haplotype_ids = [haplotypes_by_name["ε2"].id, haplotypes_by_name["ε3"].id],
        completeness = "COMPLETE"
    ),
    "ε3/ε2": models.Genotype(
        haplotype_ids = [haplotypes_by_name["ε3"].id, haplotypes_by_name["ε2"].id],
        completeness = "COMPLETE"
    ),
    "ε4/ε4": models.Genotype(
        haplotype_ids = [haplotypes_by_name["ε4"].id, haplotypes_by_name["ε4"].id],
        completeness = "COMPLETE"
    ),
}

for n, h in genotypes_by_name.items():
    h.id = computed_id(h)
    identifiers[h.id].add(ir_to_id(models.Identifier(namespace=APOE_NAMESPACE, accession=n)))

In [16]:
serialize(genotypes_by_name["ε4/ε4"])

'<Genotype|COMPLETE|[VMC:GH_xk_4sKZKfwD7ol3H89mDShrBT3dfu5Aq;VMC:GH_xk_4sKZKfwD7ol3H89mDShrBT3dfu5Aq]>'

In [17]:
opp(genotypes_by_name["ε2/ε3"])

{
    "completeness": "COMPLETE",
    "haplotype_ids": [
        "VMC:GH_PZt2IfE2__4A_J4inA1Ehkyia5sir3FK",
        "VMC:GH_VIAXr7XHqVGdcaFeDRHizjXfG83xYEg6"
    ],
    "id": "VMC:GG_Y8F4nhC0VAMK_uVrC2DjLcQ_Eed03ahq"
}


In [18]:
opp(genotypes_by_name["ε3/ε2"])

{
    "completeness": "COMPLETE",
    "haplotype_ids": [
        "VMC:GH_VIAXr7XHqVGdcaFeDRHizjXfG83xYEg6",
        "VMC:GH_PZt2IfE2__4A_J4inA1Ehkyia5sir3FK"
    ],
    "id": "VMC:GG_Y8F4nhC0VAMK_uVrC2DjLcQ_Eed03ahq"
}


## Bundle Serialization, Validation, and Roundtripping

In [19]:
bundle = models.Vmcbundle(
    meta=models.Meta(
            generated_at=datetime.datetime.isoformat(datetime.datetime.now()),
            vmc_version=0,
        ),
    locations = {o.id: o.as_dict() for o in locations_by_name.values()},
    alleles = {o.id: o.as_dict() for o in alleles_by_name.values()},
    haplotypes = {o.id: o.as_dict() for o in haplotypes_by_name.values()},
    genotypes = {o.id: o.as_dict() for o in genotypes_by_name.values()},
    identifiers = {n: list(irs) for n, irs in identifiers.items()}
)

In [20]:
opp(bundle)

{
    "alleles": {
        "VMC:GA_AnJl99FJB5tNPupduz8I4R8CCuwCpIY0": {
            "id": "VMC:GA_AnJl99FJB5tNPupduz8I4R8CCuwCpIY0",
            "location_id": "VMC:GL_L1IS6jOwSUsOpKihGRcqxHul1IwbV-1s",
            "state": "T"
        },
        "VMC:GA_BqFKgQ350SUi4-uaGCvMM1ag7z0dNmvC": {
            "id": "VMC:GA_BqFKgQ350SUi4-uaGCvMM1ag7z0dNmvC",
            "location_id": "VMC:GL_heG_PQopyF0Vgi4zH6yiLf3diyfyJcV3",
            "state": "T"
        },
        "VMC:GA__8rLiy7YkQDNy-t536RpVFGxIDiWLr6J": {
            "id": "VMC:GA__8rLiy7YkQDNy-t536RpVFGxIDiWLr6J",
            "location_id": "VMC:GL_heG_PQopyF0Vgi4zH6yiLf3diyfyJcV3",
            "state": "C"
        },
        "VMC:GA_zsJuMckKGajqHCl16sxKQJtBMjGrFHHZ": {
            "id": "VMC:GA_zsJuMckKGajqHCl16sxKQJtBMjGrFHHZ",
            "location_id": "VMC:GL_L1IS6jOwSUsOpKihGRcqxHul1IwbV-1s",
            "state": "C"
        }
    },
    "genotypes": {
        "VMC:GG_Pv97fICMeVRmowtCwioFpoFmrsOkZ7es": {
            "completene

### Validate against schema

In [21]:
s = bundle.serialize()  # same as above opp(bundle), but not pretty printed

In [22]:
schema = json.load(open(schema_path))
jsonschema.validate(bundle.as_dict(), schema)

### Verify that bundle roundtrips to same structure

In [23]:
bundle_round_trip = models.Vmcbundle(**json.loads(s))

In [24]:
bundle == bundle_round_trip

True

### Save bundle
This will be used in the VMC Bundle Example

In [25]:
open("ApoE Example.vmc.json", "w").write(s)

3418